#Overview
This file shows how to the following steps:

Reading raw files

Filtering

Channel interpolation

Reading events

Re-referencing

ICA

Fitting ICA

Segmentation






# Installing libraries

In [3]:
#I changed the name for each participant and defined an ID to avoid hardcoding. By changing these 2 parameters, the script can be run for any file.
fname='01_Participant_.EDF'
p_id='1-1'

The first step is to install the libraries.

In [2]:
!pip install mne
!pip install numpy
import numpy as np
import pandas
import matplotlib.pyplot as plt
!pip install pymatreader

In [7]:
#I used the files on my google drive. So it needs to be imported.
from google.colab import drive
drive.mount('gdrive')
import mne
import matplotlib.pyplot as plt

Mounted at gdrive


# Choosing the File

In [11]:
#changing the directory to the folder, where your files are.
#cd /content//gdrive/MyDrive/

# Reading raw files

In [ ]:
# raw data with different formats can be read with MNE.
raw=mne.io.read_raw_edf(fname, preload=True)
#raw=mne.io.read_raw_eeglab(fname, preload=True)
#raw=mne.io.read_raw_brainvision(fname, preload=True)




Extracting EDF parameters from /content/gdrive/MyDrive/DrivingData/EEGData/Raw_Data/01_Participant_2022-10-14_13-02-50_Segment_0.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 406047  =      0.000 ...   793.061 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 16897 samples (33.002 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.4s
<ipython-input-77-4fde8296a5d5>:11: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw_filt_int = raw_filt.copy().interpolate_bads(reset_bads=False)


#Montage information
you chosse the montage tyoe of your eeg cap.

In [ ]:
montage='standard_1020'
raw.set_montage(montage, on_missing='ignore')

#Filter
I filtered data between the range of 0.1 Hz and 40 Hz.


In [ ]:
raw_filt=raw.copy().filter(0.1,40)


#interpolation  for bad channels


In [ ]:
raw_filt_int = raw_filt.copy().interpolate_bads(reset_bads=False)

#Events

In [ ]:
#The actual events in each file were 92, but some duplicate events were recorded. The event ids for the true events was 4.

events, event_ids = mne.events_from_annotations(raw_filt_int)
events2=events[0];
for i in range (1,len(events)):
  if events[i,2]==4 and events[i-1,2]!=4:
    events2= np.vstack([events2, events[i]])
Finalevents=events2[1:,:]
conditions = {'Straight':0, 'Left':1, 'Right':2 }
#This file shows the order of events. 0 , a , and 2 demonstrate the three classes.
labels=[1,	0,	1,	0,	2,	0,	1,	0,	2,	0,	2,	0,	2,	0,	1,	0,	1,	0,	2,	0,	2,	0,	1,	0,	2,	0,	2,	0,	1,	0,	1,	0,	1,	0,	2,	0,	2,	0,	1,	0,	2,	0,	2,	0,	2,	0,	1,	0,	2,	0,	1,	0,	1,	0,	2,	0,	2,	0,	2,	0,	1,	0,	1,	0,	1,	0,	2,	0,	2,	0,	1,	0,	2,	0,	2,	0,	1,	0,	1,	0,	1,	0,	2,	0,	1,	0,	2,	0,	2,	0,	1,	0];
Finalevents[:,2]=labels;

In [ ]:
len(Finalevents)

92

** It is also possible to seperate event file for each condition.**

In [ ]:
#seperate conditions
str_events=[0,0,0];
left_events=[0,0,0];
right_events=[0,0,0];
for i in range(0 , 92):
  if labels[i]==0:
    str_events=np.vstack([str_events, Finalevents[i]]);
  elif labels[i]==1:
    left_events=np.vstack([left_events, Finalevents[i]]);
  else:
    right_events=np.vstack([right_events, Finalevents[i]]);

str_events=str_events[1:,:];
left_events=left_events[1:,:];
right_events=right_events[1:,:]

#Re-referencing
The new reference can be the average of all channels, or using the two mastoid electrodes, or any other electrode of interest.

In [ ]:
raw_filt_int_avgref=raw_filt_int.copy().set_eeg_reference(ref_channels='average');

#raw_filt_int_mastoidref=raw_filt_int.copy().set_eeg_reference(ref_channels=['M1','M2']);

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


#ICA

#Creating epochs for tunning ica parameters only.
It is better to fit the ICA on raw data before the actual segmentation.

In [ ]:
conditions = {'Straight':0, 'Left':1, 'Right':2 }
tmin=-0.200
tmax=4
epochs=mne.Epochs(raw_filt_int_avgref.copy(), Finalevents, event_id=conditions, tmin=tmin, tmax=tmax, baseline=None, preload=True)


Not setting metadata
92 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 92 events and 2151 original time points ...
0 bad epochs dropped


In [ ]:
!pip install -U autoreject
from autoreject import get_rejection_threshold
reject = get_rejection_threshold(epochs);

Estimating rejection dictionary for eeg


In [ ]:
# ICA parameters
tstep = 1.96
random_state = 42   # ensures ICA is reproducable each time it's run
ica_n_components = .99     # Specify n_components as a decimal to set % explained variance

In [ ]:
ica = mne.preprocessing.ICA(n_components=ica_n_components, random_state=random_state)
ica.fit(raw_filt_int_avgref.copy(), reject=reject, tstep=tstep);

Fitting ICA to data using 64 channels (please be patient, this may take a while)
    Rejecting  epoch based on EEG : ['EOG']
Artifact detected in [6024, 7028]
    Rejecting  epoch based on EEG : ['EOG']
Artifact detected in [60240, 61244]
    Rejecting  epoch based on EEG : ['Fp1', 'Fpz', 'Fp2', 'EOG']
Artifact detected in [66264, 67268]
    Rejecting  epoch based on EEG : ['Fp1', 'Fpz', 'Fp2', 'EOG']
Artifact detected in [67268, 68272]
    Rejecting  epoch based on EEG : ['Fp1', 'Fpz', 'Fp2', 'EOG']
Artifact detected in [68272, 69276]
    Rejecting  epoch based on EEG : ['Fp1', 'Fpz', 'Fp2', 'EOG']
Artifact detected in [69276, 70280]
    Rejecting  epoch based on EEG : ['Fp1', 'Fpz', 'Fp2', 'AF7']
Artifact detected in [70280, 71284]
    Rejecting  epoch based on EEG : ['Fp1', 'Fpz', 'Fp2', 'EOG', 'AF8']
Artifact detected in [72288, 73292]
    Rejecting  epoch based on EEG : ['Fpz', 'Fp2', 'EOG', 'AF8']
Artifact detected in [73292, 74296]
    Rejecting  epoch based on EEG : ['Fp1', 'Fp

In [ ]:
ica.save('raw_filt_int_ica_'+ p_id+ '.fif',overwrite=True);

Overwriting existing file.
Writing ICA solution to /content/gdrive/MyDrive/DrivingData/EEGData/Raw_Data/raw_filt_int_ica_1-1.fif...


<ipython-input-99-681dd0346ee4>:1: RuntimeWarning: This filename (raw_filt_int_ica_1-1.fif) does not conform to MNE naming conventions. All ICA files should end with -ica.fif, -ica.fif.gz, _ica.fif or _ica.fif.gz
  ica.save('raw_filt_int_ica_'+ p_id+ '.fif',


In [ ]:
raw_filt_int_avgref_ica=ica.apply(raw_filt_int_avgref.copy(), exclude=bad_idx)

Applying ICA to Raw instance
    Transforming to ICA space (43 components)
    Zeroing out 0 ICA components
    Projecting back using 64 PCA components


#Epochs
Creating epochs and saving them

In [ ]:
epochs_all=mne.Epochs(raw_filt_int_avgref_ica.copy(), Finalevents,  tmin=tmin, tmax=tmax, baseline=None, preload=True)
epochs_str=mne.Epochs(raw_filt_int_avgref_ica.copy(), str_events,  tmin=tmin, tmax=tmax, baseline=None, preload=True)
epochs_left=mne.Epochs(raw_filt_int_avgref_ica.copy(), left_events,  tmin=tmin, tmax=tmax, baseline=None, preload=True)
epochs_right=mne.Epochs(raw_filt_int_avgref_ica.copy(), right_events, tmin=tmin, tmax=tmax, baseline=None, preload=True)
epochs_str.save('epochs_str_'+ p_id+ '.fif', overwrite=True);
epochs_left.save('epochs_left_'+ p_id+ '.fif', overwrite=True);
epochs_right.save('epochs_right_'+ p_id+ '.fif', overwrite=True);
epochs_all.save('epochs_all_'+ p_id+ '.fif', overwrite=True);

Not setting metadata
92 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 92 events and 2151 original time points ...
0 bad epochs dropped
Not setting metadata
46 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 46 events and 2151 original time points ...
0 bad epochs dropped
Not setting metadata
22 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 22 events and 2151 original time points ...
0 bad epochs dropped
Not setting metadata
24 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 24 events and 2151 original time points ...
0 bad epochs dropped
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.


<ipython-input-101-ed26fb5f1773>:5: RuntimeWarning: This filename (epochs_str_1-1.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_str.save('epochs_str_'+ p_id+ '.fif', overwrite=True);
<ipython-input-101-ed26fb5f1773>:6: RuntimeWarning: This filename (epochs_left_1-1.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_left.save('epochs_left_'+ p_id+ '.fif', overwrite=True);
<ipython-input-101-ed26fb5f1773>:7: RuntimeWarning: This filename (epochs_right_1-1.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_right.save('epochs_right_'+ p_id+ '.fif', overwrite=True);


Overwriting existing file.


<ipython-input-101-ed26fb5f1773>:8: RuntimeWarning: This filename (epochs_all_1-1.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_all.save('epochs_all_'+ p_id+ '.fif', overwrite=True);
